In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_train = pd.read_csv('../input/train.csv')

In [ ]:
df_train.shape

In [ ]:
y = df_train.label
X = df_train.drop(columns="label")
X.head()

In [ ]:
y.head()

In [ ]:
plt.imshow(np.reshape(X.values[0], newshape=[28,28]) );

In [ ]:
X = X/255.   # V.V.I: Normalization

In [ ]:
X.values[1].shape

# LSTM

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Bidirectional, Dense, Concatenate, GlobalMaxPooling1D

In [ ]:
ip_tensor = Input(shape=(28,28))    # Instantiate Keras tensor

In [ ]:
t = Bidirectional(LSTM(units=15, activation="relu", return_sequences=True))(ip_tensor)

In [ ]:
t = GlobalMaxPooling1D()(t)

In [ ]:
ip_tensor_T = Input(shape=(28,28))
t_T = Bidirectional(LSTM(15, activation="relu", return_sequences=True))(ip_tensor_T)

In [ ]:
t_T = GlobalMaxPooling1D()(t_T)

In [ ]:
t_T.shape

In [ ]:
t_concat = Concatenate(axis=-1)([t,t_T])

In [ ]:
t_concat.shape

In [ ]:
L1 = Dense(units=10, activation="softmax")(t_concat)

In [ ]:
my_model = Model([ip_tensor, ip_tensor_T], L1)

# Training bit

In [ ]:
from keras.optimizers import Adam

In [ ]:
my_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
my_model.summary()

# Initiate training

In [ ]:
X_reshaped = X.values.reshape([-1,28,28])

In [ ]:
#plt.imshow(X_reshaped[20,:,:])
plt.imshow(np.rot90(X_reshaped, axes=(1,2))[20,:,:])

In [ ]:
trace = my_model.fit([X_reshaped, np.rot90(X_reshaped, axes=(1,2))], y, batch_size=300,
                     epochs=50,validation_split=0.3)

# Prediction

In [ ]:
df_sample_submission = pd.read_csv('../input/sample_submission.csv')
df_test = pd.read_csv('../input/test.csv')

In [ ]:
df_sample_submission

In [ ]:
df_test = df_test/255.
test_reshaped = df_test.values.reshape([-1,28,28])

In [ ]:
pred = my_model.predict([test_reshaped, np.rot90(test_reshaped, axes=(1,2))])

In [ ]:
df_test_pred = pd.DataFrame({"ImageId":range(1,pred.shape[0]+1),  "Label":pred.argmax(axis=1)})
df_test_pred

In [ ]:
df_test_pred.to_csv("submission.csv")